# Imports

In [3]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
export_dir = os.getcwd()
from pathlib import Path
from scipy import sparse
from os import path
import shap

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
import pickle
import ipynb
import importlib
import warnings

from torch.nn import Softmax
softmax = nn.Softmax()

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
data_name = "ML1M" ### Can be ML1M, Yahoo, Pinterest
recommender_name = "MLP" 
DP_DIR = Path("processed_data", data_name) 
export_dir = Path(os.getcwd())
files_path = Path(export_dir.parent, DP_DIR)
checkpoints_path = Path(export_dir.parent, "checkpoints")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
output_type_dict = {
    "VAE":"multiple",
    "MLP":"single",
    "NCF": "single"}

num_users_dict = {
    "ML1M":6037,
    "Yahoo":13797, 
    "Pinterest":19155}

num_items_dict = {
    "ML1M":3381,
    "Yahoo":4604, 
    "Pinterest":9362}


recommender_path_dict = {
    ("ML1M","VAE"): Path(checkpoints_path, "VAE_ML1M_0.0007_128_10.pt"),
    ("ML1M","MLP"):Path(checkpoints_path, "MLP1_ML1M_0.0076_256_7.pt"),
    ("ML1M","NCF"):Path(checkpoints_path, "NCF_ML1M_5e-05_64_16.pt"),
    
    ("Yahoo","VAE"): Path(checkpoints_path, "VAE_Yahoo_0.0001_128_13.pt"),
    ("Yahoo","MLP"):Path(checkpoints_path, "MLP2_Yahoo_0.0083_128_1.pt"),
    ("Yahoo","NCF"):Path(checkpoints_path, "NCF_Yahoo_0.001_64_21_0.pt"),
    
    ("Pinterest","VAE"): Path(checkpoints_path, "VAE_Pinterest_12_18_0.0001_256.pt"),
    ("Pinterest","MLP"):Path(checkpoints_path, "MLP_Pinterest_0.0062_512_21_0.pt"),
    ("Pinterest","NCF"):Path(checkpoints_path, "NCF2_Pinterest_9e-05_32_9_10.pt"),}


hidden_dim_dict = {
    ("ML1M","VAE"): None,
    ("ML1M","MLP"): 32,
    ("ML1M","NCF"): 8,

    ("Yahoo","VAE"): None,
    ("Yahoo","MLP"):32,
    ("Yahoo","NCF"):8,
    
    ("Pinterest","VAE"): None,
    ("Pinterest","MLP"):512,
    ("Pinterest","NCF"): 64,
}

In [6]:
from ipynb.fs.defs.help_functions import *
importlib.reload(ipynb.fs.defs.help_functions)
from ipynb.fs.defs.help_functions import *

### Import MLP recommender

In [7]:
from ipynb.fs.defs.recommenders_architecture import *
importlib.reload(ipynb.fs.defs.recommenders_architecture)
from ipynb.fs.defs.recommenders_architecture import *

# SHAP

### MLP Wrapper for SHAP

In [8]:
class MLPWrapper(MLP):
    def __init__(self, hidden_size, cluster_to_items, **kw):
        super().__init__(hidden_size=hidden_size, device=device, num_items=num_items)
        self.cluster_to_items = cluster_to_items
        self.items_array = items_array
        self.device = device
        self.num_items = num_items
        
    def preprocess(self, batch):
        items = batch[:, 0]
        clusters = batch[:, 1:]
        n_clusters = clusters.shape[1]

        items_tensor = torch.Tensor(self.items_array[items]).to(self.device)
        user_tensor = torch.zeros((len(batch), self.num_items), dtype=torch.float).to(self.device)

        for cluster in range(n_clusters):
            cluster_indices = torch.tensor(clusters[:, cluster], dtype=torch.float).to(self.device)
            user_tensor[:, self.cluster_to_items[cluster]] = cluster_indices.unsqueeze(1)

        return user_tensor, items_tensor

    def forward(self, batch):
        batch_size = 256  
        outputs = []
        for i in range(0, len(batch), batch_size):
            mini_batch = batch[i:i+batch_size]
            user_tensor, items_tensor = self.preprocess(mini_batch)
            output = super().forward(user_tensor, items_tensor)
            outputs.append(torch.diag(output).detach().cpu().numpy())
        return np.concatenate(outputs)

### Read data

In [9]:
train_data = pd.read_csv(Path(files_path,f'train_data_{data_name}.csv'), index_col=0)
test_data = pd.read_csv(Path(files_path,f'test_data_{data_name}.csv'), index_col=0)
train_array = train_data.to_numpy()
test_array = test_data.to_numpy()

In [10]:
#users' ids in the test dataset
row_test_indices = np.arange(test_array.shape[0]) + train_array.shape[0]

## Create / Load top recommended item dict 

In [11]:
with open(Path(files_path,f'pop_dict_{data_name}.pkl'), 'rb') as f:
    pop_dict = pickle.load(f)
pop_array = np.zeros(len(pop_dict))
for key, value in pop_dict.items():
    pop_array[key] = value

output_type = output_type_dict[recommender_name] ### Can be single, multiple
num_users = num_users_dict[data_name] 
num_items = num_items_dict[data_name] 

items_array = np.eye(num_items)
all_items_tensor = torch.Tensor(items_array).to(device)

hidden_dim = hidden_dim_dict[(data_name,recommender_name)]
recommender_path = recommender_path_dict[(data_name,recommender_name)]

kw_dict = {'device':device,
          'num_items': num_items,
           'num_features': num_items, 
            'demographic':False,
          'pop_array':pop_array,
          'all_items_tensor':all_items_tensor,
          'items_array':items_array,
          'output_type':output_type,
          'recommender_name':recommender_name}

In [12]:
def load_recommender():
    if recommender_name=='MLP':
        recommender = MLP(hidden_dim, **kw_dict)
    elif recommender_name=='VAE':
        if data_name == "Pinterest":
            recommender = VAE(Pinterest_VAE_config, **kw_dict)
        else:
            recommender = VAE(VAE_config, **kw_dict)
    elif recommender_name=='NCF':
        MLP_temp = MLP_model(hidden_size=hidden_dim, num_layers=3, **kw_dict)
        GMF_temp = GMF_model(hidden_size=hidden_dim, **kw_dict)
        recommender = NCF(factor_num=hidden_dim, num_layers=3, dropout=0.5, model= 'NeuMF-pre', GMF_model= GMF_temp, MLP_model=MLP_temp, **kw_dict)
    
    recommender_checkpoint = torch.load(Path(checkpoints_path, recommender_path))
    recommender.load_state_dict(recommender_checkpoint)
    recommender.eval()
    for param in recommender.parameters():
        param.requires_grad= False
        
    return recommender


recommender = load_recommender()
optimizer = torch.optim.Adam(recommender.parameters(), lr=0.001)

In [13]:
create_dicts = True # True if it is the first time running the notebook for the specific recommender and data set

if create_dicts:
    top1_train = {}
    top1_test = {}  
    for i in range(train_array.shape[0]):
        user_index = int(train_data.index[i])
        user_tensor = torch.Tensor(train_array[i]).to(device)
        top1_train[user_index] = int(get_user_recommended_item(user_tensor, recommender, **kw_dict))
    for i in range(test_array.shape[0]):
        user_index = int(test_data.index[i])
        user_tensor = torch.Tensor(test_array[i]).to(device)
        top1_test[user_index] = int(get_user_recommended_item(user_tensor, recommender, **kw_dict))
        
    with open(Path(files_path,f'top1_train_{data_name}_{recommender_name}.pkl'), 'wb') as f:
        pickle.dump(top1_train, f)
    with open(Path(files_path,f'top1_test_{data_name}_{recommender_name}.pkl'), 'wb') as f:
        pickle.dump(top1_test, f)
else:
    with open(Path(files_path,f'top1_train_{data_name}_{recommender_name}.pkl'), 'rb') as f:
        top1_train = pickle.load(f)
    with open(Path(files_path,f'top1_test_{data_name}_{recommender_name}.pkl'), 'rb') as f:
        top1_test = pickle.load(f)

### Clustering

In [14]:
K = 100
u_train = torch.tensor(train_array).float()
v_train = all_items_tensor

In [15]:
np.random.seed(3)
# Cluster items using k-means
from sklearn.cluster import KMeans
import numpy as np
k = 10

kmeans = KMeans(n_clusters=k)
clusters = kmeans.fit_predict(np.transpose(u_train))

In [16]:
item_clusters = kmeans.predict(np.transpose(u_train))

# Create mapping from items to clusters
item_to_cluster = {}
# Create mapping from clusters to items
cluster_to_items = {}
for i, cluster in enumerate(item_clusters):
    item_to_cluster[i] = cluster
    if(cluster not in cluster_to_items.keys()):
        cluster_to_items[cluster] = []
    cluster_to_items[cluster].append(i)

In [17]:
u_test = torch.tensor(test_array).float()

In [18]:
user_to_clusters = np.zeros((u_test.shape[0],10))

In [19]:
for i in cluster_to_items.keys():
    user_to_clusters[:,i] = np.sum(u_test.cpu().detach().numpy().T[cluster_to_items[i]], axis=0)

In [20]:
user_to_clusters_bin =  np.where(user_to_clusters > 0, 1, 0)

In [21]:
user_to_clusters_train = np.zeros((u_train.shape[0],10))

In [22]:
user_to_clusters_test = np.zeros((u_test.shape[0],10))

In [23]:
default_value = 0
target_items_test = list(top1_test.values())
target_items_train = list(top1_train.values())

In [24]:
for i in cluster_to_items.keys():
    user_to_clusters_train[:,i] = np.sum(u_train.cpu().detach().numpy().T[cluster_to_items[i]], axis=0)

In [25]:
user_to_clusters_train_bin = np.where(user_to_clusters_train > 0, 1, 0)

In [26]:
col2 = list(top1_train.values())
input_train_array = np.insert(user_to_clusters_train_bin, 0, col2, axis=1).astype(int)

In [27]:
for i in cluster_to_items.keys():
    user_to_clusters_test[:,i] = np.sum(u_test.cpu().detach().numpy().T[cluster_to_items[i]], axis=0)

In [28]:
user_to_clusters_test_bin = np.where(user_to_clusters_test > 0, 1, 0)

In [29]:
col2 = list(top1_test.values())
input_test_array= np.insert(user_to_clusters_test_bin, 0, col2, axis=1).astype(int)

In [30]:
wrap_model =  MLPWrapper(hidden_dim, cluster_to_items, **kw_dict)

### SHAP

In [31]:
K = 50

In [32]:
sampled_subset = shap.sample(input_train_array,K)

In [33]:
explainer = shap.KernelExplainer(wrap_model, sampled_subset)

In [ ]:
shap_values_test = explainer.shap_values(input_test_array)

  9%|▉         | 107/1208 [24:09<9:54:14, 32.38s/it]

In [ ]:
col1 = row_test_indices
input_test_array = np.insert(shap_values_test[:, 1:], 0, col1,axis=1)

In [ ]:
with open(Path(files_path,f'item_to_cluster_{recommender_name}_{data_name}.pkl'), 'wb') as f:
    pickle.dump(item_to_cluster, f)

In [ ]:
with open(Path(files_path,f'shap_values_{recommender_name}_{data_name}.pkl'), 'wb') as f:
    pickle.dump(input_test_array, f)